In [1]:
from cmapPy.pandasGEXpress.parse import parse
import pandas as pd
import numpy as np
import time

In [2]:
ref_data = parse("C:/Users/ngomi/Downloads/Documents/data.gctx").data_df
ref_data.index = ref_data.index.astype(int)
gene_info = pd.read_csv("../data/gene_info.txt", delimiter="\t")
gene_dict = {}
for i in range(len(gene_info.index)):
    gene_dict[str(gene_info["gene_symbol"][i])] = gene_info["gene_id"][i]

In [3]:
up_gene_file = "../data/Example_up_genes.txt"
down_gene_file = "../data/Example_down_genes.txt"

def read_genes(file_name):
    with open(file_name) as file:
        return [gene.strip() for gene in file.readlines()]

up_genes = read_genes(up_gene_file)
down_genes = read_genes(down_gene_file)
n_expression = len(ref_data.columns)
n_ref_gene = len(ref_data.index)

up_genes = [gene_dict[i] for i in up_genes if i in gene_dict]
down_genes = [gene_dict[i] for i in down_genes if i in gene_dict]

In [6]:
from pandas import DataFrame
nearest_neighbor = np.loadtxt("../distance_files/nearest_neighbor_cosine.txt").astype(int)

def calc_es_score(ref_df, cid, gene_list):
    gene_indexes = ref_df.index.isin(gene_list)
    ns = len(gene_list)
    nr = ref_df.loc[gene_indexes, cid].sum()
    cumsum_score = np.zeros(n_ref_gene)
    cumsum_score[gene_indexes] = ref_df.loc[gene_indexes, cid] / nr
    cumsum_score[~gene_indexes] = -1 / (n_ref_gene - ns)
    scores = np.cumsum(cumsum_score)
    min_score = np.min(scores)
    max_score = np.max(scores)
    return max_score if abs(max_score) > abs(min_score) else min_score

def calc_connectivity_score(up_gene, down_gene, data, cid):
    df = data[cid]
    df = pd.DataFrame(df.sort_values(ascending=False)).abs()
    up_es_score = calc_es_score(df, cid, up_gene)
    down_es_score = calc_es_score(df, cid, down_gene)
    return (up_es_score - down_es_score) / 2 if up_es_score * down_es_score < 0 else 0
    
def cmap(data, up_gene_list, down_gene_list):
    c_scores = []
    check = set()
    proceeded = []
    for i, cid in enumerate(data.columns):
        if i not in check:
            proceeded.append(cid)
            c_score = calc_connectivity_score(up_gene_list, down_gene_list, data, cid)
            c_scores.append(c_score)
            if c_score == 0:
                check.update(nearest_neighbor[i,:10])
    return pd.DataFrame({"expression": proceeded, "c_score": c_scores})

def cmap_original(data, up_gene_list, down_gene_list):
    c_scores = []
    for cid in data.columns:
        c_score = calc_connectivity_score(up_gene_list, down_gene_list, data, cid)
        c_scores.append(c_score)
    return pd.DataFrame({"expression": data.columns, "c_score": c_scores})

In [7]:
result = cmap(ref_data, up_genes, down_genes)


In [21]:
result['expression'].tail(10).values

array(['TA.OE015_A549_96H:TRCN0000474888:-666',
       'TA.OE015_A549_96H:TRCN0000474892:-666',
       'TA.OE015_A549_96H:TRCN0000475257:-666',
       'TA.OE015_A549_96H:TRCN0000475363:-666',
       'TA.OE015_A549_96H:TRCN0000476333:-666',
       'TA.OE015_A549_96H:TRCN0000476570:-666',
       'TA.OE015_A549_96H:TRCN0000480075:-666',
       'TA.OE015_A549_96H:TRCN0000480395:-666',
       'TA.OE015_A549_96H:TRCN0000481366:-666',
       'TA.OE015_A549_96H:TRCN0000489894:-666'], dtype=object)

In [30]:
result_original = cmap_original(ref_data, up_genes, down_genes)

In [35]:
result = result.sort_values(by=["c_score"], ascending=False)
result_original = result_original.sort_values(by=["c_score"], ascending=False)

In [37]:
result

,expression,c_score
11056,OEB006_HT29_96H:BRDN0000412056:-666,0.530904
213,HSF002_HEK293T_48H:CMAP-HSF-ALK:200,0.497368
16892,TA.OE012_H1299_96H:BRDN0000553389:-666,0.486682
17023,TA.OE012_SALE_96H:BRDN0000553411:-666,0.471487
4447,OEB002_HCC515_96H:BRDN0000403123:-666,0.471198
...,...,...
16981,TA.OE012_SALE_96H:BRDN0000552530:-666,-0.386532
16694,TA.OE012_A549_96H:TRCN0000474939:-666,-0.390880
17670,TA.OE014_A549_96H:BRDN0000554117:-666,-0.397609
16712,TA.OE012_AALE_96H:BRDN0000552448:-666,-0.415773


In [36]:
def count(list_1, list_2):
    count = 0
    for i in list_1:
        if i in list_2: count += 1
    return count/(len(list_1) * 1.0)

mimix_res = list(result[:25]["expression"])
mimix_res_ori = list(result_original[:25]["expression"])
print(count(mimix_res, mimix_res_ori))

rev_res = list(result[-25:]["expression"])
rev_res_ori = list(result_original[-25:]["expression"])
print(count(rev_res, rev_res_ori))

0.64
0.52


In [42]:
result

,expression,c_score
7849,OEB006_HT29_96H:BRDN0000412056:-666,0.530904
180,HSF002_HEK293T_48H:CMAP-HSF-ALK:200,0.497368
3407,OEB002_HCC515_96H:BRDN0000403123:-666,0.471198
278,HSF006_HEK293T_48H:CMAP-HSF-ALK_T1151M:200,0.462748
11757,TA.OE012_SALE_96H:BRDN0000553456:-666,0.455296
...,...,...
12164,TA.OE015_A549_96H:BRDN0000552890:-666,-0.382085
11873,TA.OE013_AALE_96H:BRDN0000552634:-666,-0.386097
11565,TA.OE012_A549_96H:TRCN0000474939:-666,-0.390880
12138,TA.OE014_A549_96H:BRDN0000554117:-666,-0.397609


In [66]:
import random
def generate_data(n = 1000):
    input_file = open("../data/input.txt", "a+")
    out_file = open("../data/output.txt", "a+")
    for _ in range(n):
        n_up_gene = random.randint(10, 150)
        n_down_gene = random.randint(10, 150)
        selected_genes = random.choices(gene_info["gene_id"], k=n_up_gene+n_down_gene)
        up_genes = selected_genes[:n_up_gene]
        down_genes = selected_genes[n_up_gene:]
        _input = []
        for g in gene_info["gene_id"]:
            if g in up_genes:
                _input.append(1)
            elif g in down_genes:
                _input.append(-1)
            else: _input.append(0)
        c_score = cmap(up_genes, down_genes)
        input_str = " ".join([str(i) for i in _input]) + "\n"
        output_str = " ".join([str(i) for i in c_score["c_score"]]) + "\n"
        input_file.write(input_str)
        out_file.write(output_str)
    input_file.close()
    out_file.close()
        

In [67]:
generate_data(100)

In [39]:
import numpy as np
from cmapPy.pandasGEXpress.parse import parse
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cdist, squareform
from joblib import Parallel, delayed
import os

# Assuming 'data' is your dataset
# Use float32 if applicable to reduce memory usage
data = parse("C:/Users/ngomi/Downloads/Documents/data.gctx").data_df
data = data.astype('float32').to_numpy().transpose()
output_directory = "../distance_files"
os.makedirs(output_directory, exist_ok=True)


In [40]:
def calculate_and_save_distances(i):
    start_row = i * 10000
    end_row = min((i+1)*10000, n_rows)
    subset_data = data[start_row: end_row]
    distances = cdist(subset_data, data, metric='minkowski')
    top_indices = np.argsort(distances, axis=1)[:, 1:51]
    np.savetxt(f"{output_directory}/nearest_neighbor_{i}.txt", top_indices, fmt='%d')

# Set the number of CPU cores
n_cores = -1
n_rows = len(data)
Parallel(n_jobs=n_cores)(
    delayed(calculate_and_save_distances)(i) for i in range(n_rows//10000 + 1)
)

KeyboardInterrupt: 

In [6]:
def file_index(file):
    dot_index = file.index(".")
    return int(file[17:dot_index])

In [17]:
from os import listdir
from os.path import isfile, join
files = [f for f in listdir(output_directory) if isfile(join(output_directory, f))]
files.sort(key=lambda f: file_index(f))

In [19]:
nearest_neighbor = []
for file in files:
    with open(output_directory + "/" + file) as f:
        for line in f:
            nearest_neighbor.append(list(map(int, line.split(" "))))

In [20]:
np.savetxt(f"{output_directory}/nearest_neighbor.txt", nearest_neighbor, fmt='%d')

In [5]:
result = {'up_genes': [9293, 5996, 136, 1029, 56934, 1365, 10284, 10287, 6899, 7762, 1933, 916, 5636, 11262, 56993, 79738, 51117, 9978, 56180, 54800, 3426, 89870, 8125], 'down_genes': [64860, 10397, 114876, 3188, 2134, 10905, 64699, 55717, 375, 8778, 247, 2841, 8539, 2147, 5081, 11313, 26011, 6520, 114897, 51663, 91683, 2745, 5167, 10109, 27020, 79929, 79864, 6620, 3371, 4868, 54472, 393, 54799, 4736, 8569, 4701, 7398, 79849, 23082, 80185, 8473, 5251, 57470, 51442, 9454, 330, 80342, 60370, 81545, 5354, 392, 6461, 11160, 7223, 5803, 6121, 3672, 25790, 3093, 5148, 23234, 6773, 26100, 4552, 11191, 5564, 373156, 6844, 23522, 8563, 23158, 10365, 29777, 3757, 9200, 26508, 4836, 11212, 57282, 80823, 54463, 10313, 55863, 56834, 51145, 549, 26155, 80185, 51567, 2806, 1381, 63932, 2677, 10951, 5861, 9923, 51329, 654, 23038, 27094, 9726, 55196, 1521, 8492, 11051, 6689, 4908, 146227, 8942, 3101, 6591, 6007, 6490, 55052, 414, 27158, 9677, 58513, 9367, 54520, 55334, 11198, 2304], 'original_mimic_result': ['OEB005_HCC515_96H:BRDN0000410382:-666', 'OEB004_MCF7_96H:BRDN0000406521:-666', 'TA.OE013_H1299_96H:TRCN0000476570:-666', 'OEB001_PC3_96H:BRDN0000398922:-666', 'MDVL001_293T_72H:2010_S54F:200', 'OEB001_HA1E_96H:BRDN0000399407:-666', 'OEB004_HA1E_96H:BRDN0000407198:-666', 'OEB002_A549_96H:BRDN0000401138:-666', 'OEB004_A375_96H:BRDN0000408555:-666', 'OFL002_A375_96H:M07', 'TA.OE008_HA1E_96H:BRDN0000465088:-666', 'OEB002_VCAP_96H:BRDN0000403103:-666', 'OEB001_A549_96H:BRDN0000399220:-666', 'OEB001_A549_96H:BRDN0000400194:-666', 'OEB001_A375_96H:BRDN0000399713:-666', 'OEB003_HCC515_96H:BRDN0000403865:-666', 'OEC001_A375_96H:CCSBBROAD304_02402:-666', 'TA.OE012_SALE_96H:BRDN0000553236:-666', 'OEB002_HCC515_96H:BRDN0000403216:-666', 'TA.OE012_H1299_96H:BRDN0000553964:-666', 'OEB004_HEPG2_96H:BRDN0000407849:-666', 'TA.OE003_PC3_72H:CCSBBROAD304_06132:-666', 'OEB006_PC3_96H:BRDN0000412027:-666', 'TA.OE007_A549_96H:CCSBBROAD304_01613:-666', 'OEB002_HCC515_96H:BRDN0000402081:-666', 'OEC001_A375_96H:CCSBBROAD304_00529:-666', 'OEB001_HT29_96H:BRDN0000400065:-666', 'OEC001_A375_96H:CCSBBROAD304_03149:-666', 'OEB006_VCAP_96H:BRDN0000461155:-666', 'OEB005_PC3_96H:BRDN0000409807:-666', 'TA.OE012_A549_96H:BRDN0000554024:-666', 'HSF017_HEK293T_48H:CMAP-HSF-CUL3:200', 'OEB003_A375_96H:BRDN0000405621:-666', 'MUT001_HEK293T_XH:ZC16:-666:20', 'OEC001_PC3_24H:CCSBBROAD304_01114:-666', 'OEB005_PC3_96H:BRDN0000408936:-666', 'OEB004_A375_96H:BRDN0000406499:-666', 'OEB005_HCC515_96H:BRDN0000409131:-666', 'OFL001_A375_96H:G02', 'OEB005_HCC515_96H:BRDN0000409181:-666', 'OEB003_MCF7_96H:BRDN0000405671:-666', 'OEB004_HCC515_96H:BRDN0000408778:-666', 'OEB006_HCC515_96H:BRDN0000411712:-666', 'OEB002_A549_96H:BRDN0000403019:-666', 'OEB002_VCAP_96H:BRDN0000401214:-666', 'TA.OE007_HA1E_96H:CCSBBROAD304_10689:-666', 'OEB005_PC3_96H:BRDN0000409446:-666', 'TA.OE013_A549_96H:BRDN0000560262:-666', 'OEC001_A375_96H:CCSBBROAD304_11708:-666', 'OEB002_A375_96H:O22'], 'original_mimic_score': [0.4706326321753459, 0.43893543043321526, 0.43753302194689514, 0.43467095557017466, 0.4328643869433527, 0.4258078757995851, 0.41975917542928365, 0.4100512771658337, 0.4072509526192558, 0.4070869768874521, 0.40529165753820107, 0.4040923206622825, 0.4024298900555166, 0.40233168066523034, 0.39864814105255786, 0.39542419365517406, 0.3948019545125533, 0.3944745641431307, 0.39390267463772527, 0.39201021015107773, 0.38992244625865824, 0.38823441389871677, 0.3881584688933833, 0.3868416672811867, 0.38621130696103095, 0.3855200725472467, 0.385049925742043, 0.38234401088773784, 0.38224035572174353, 0.3806047132429205, 0.3798968856536714, 0.37983597430901084, 0.3789439421176715, 0.37805167310071297, 0.37777965658586954, 0.3771142715442274, 0.3770135659646754, 0.3764793708058721, 0.37636571509465244, 0.3762027560232346, 0.375505082241598, 0.37534264378032334, 0.37502313066296367, 0.374747934306502, 0.37461988598012674, 0.37440983538954054, 0.37418317559497283, 0.3739467364651115, 0.37335075422604186, 0.37305531388623314], 'original_reverse_result': ['OEB006_HA1E_96H:BRDN0000411442:-666', 'OEB002_HCC515_96H:BRDN0000403217:-666', 'TA.OE012_SALE_96H:BRDN0000552914:-666', 'OEB001_A375_96H:BRDN0000399286:-666', 'OEB003_VCAP_96H:BRDN0000405583:-666', 'MUT002_HEK293T_XH:HS15:-666:30', 'OEC001_PC3_24H:CCSBBROAD304_01921:-666', 'MDVL001_293T_96H:7276_V30M:200', 'HSF002_HEK293T_48H:CMAP-HSF-BMPR1A:200', 'TA.OE008_A549_96H:CCSBBROAD304_15013:-666', 'OEB005_MCF7_96H:BRDN0000410382:-666', 'OEB003_HEPG2_96H:BRDN0000404273:-666', 'OEB001_MCF7_96H:BRDN0000399470:-666', 'OEB003_VCAP_96H:BRDN0000404273:-666', 'OEB004_HEPG2_96H:BRDN0000408684:-666', 'OEB002_HEPG2_96H:BRDN0000403163:-666', 'OEC001_PC3_72H:CCSBBROAD304_14640:-666', 'HSFA001_HEK293T_48H:N06', 'TA.OE013_AALE_96H:BRDN0000553555:-666', 'OEC001_A375_48H:CCSBBROAD304_07674:-666', 'OEB004_HEPG2_96H:BRDN0000408496:-666', 'TA.OE008_A549_96H:BRDN0000464814:-666', 'OEB004_A375_96H:BRDN0000407450:-666', 'TA.OE012_SALE_96H:BRDN0000553173:-666', 'OEB005_HT29_96H:C20', 'OEB005_A375_96H:BRDN0000409170:-666', 'OEB003_MCF7_96H:BRDN0000404637:-666', 'TA.OE008_A549_96H:BRDN0000465130:-666', 'OEC001_PC3_96H:CCSBBROAD304_09956:-666', 'TA.OE007_HA1E_96H:BRDN0000465049:-666', 'OEB005_A549_96H:BRDN0000409322:-666', 'MDVL001_293T_72H:114327_R221H:200', 'OEC001_PC3_96H:CCSBBROAD304_09923:-666', 'TA.OE007_AALE_96H:BRDN0000464894:-666', 'HSF027_HEK293T_48H:CMAP-HSF-CBLL1:200', 'OEB006_HT29_96H:BRDN0000411931:-666', 'OEB004_HA1E_96H:BRDN0000408626:-666', 'MUT001_HEK293T_XH:AG15-TB:-666:24', 'OEB002_A375_96H:M01', 'TA.OE010_HA1E_96H:BRDN0000560946:3', 'OEB005_PC3_96H:BRDN0000410350:-666', 'TA.OE010_HA1E_96H:TRCN0000475489:3', 'OEB005_MCF7_96H:BRDN0000408936:-666', 'OEB006_HT29_96H:BRDN0000411483:-666', 'TA.OE008_A549_96H:BRDN0000465157:-666', 'OEC001_A375_24H:CCSBBROAD304_07050:-666', 'TA.OE008_HA1E_96H:BRDN0000465151:-666', 'TA.OE015_A549_96H:BRDN0000560362:-666', 'TA.OE007_H1299_96H:CCSBBROAD304_14634:-666', 'OEB002_A375_96H:C12'], 'original_reverse_score': [0.4706326321753459, 0.43893543043321526, 0.43753302194689514, 0.43467095557017466, 0.4328643869433527, 0.4258078757995851, 0.41975917542928365, 0.4100512771658337, 0.4072509526192558, 0.4070869768874521, 0.40529165753820107, 0.4040923206622825, 0.4024298900555166, 0.40233168066523034, 0.39864814105255786, 0.39542419365517406, 0.3948019545125533, 0.3944745641431307, 0.39390267463772527, 0.39201021015107773, 0.38992244625865824, 0.38823441389871677, 0.3881584688933833, 0.3868416672811867, 0.38621130696103095, 0.3855200725472467, 0.385049925742043, 0.38234401088773784, 0.38224035572174353, 0.3806047132429205, 0.3798968856536714, 0.37983597430901084, 0.3789439421176715, 0.37805167310071297, 0.37777965658586954, 0.3771142715442274, 0.3770135659646754, 0.3764793708058721, 0.37636571509465244, 0.3762027560232346, 0.375505082241598, 0.37534264378032334, 0.37502313066296367, 0.374747934306502, 0.37461988598012674, 0.37440983538954054, 0.37418317559497283, 0.3739467364651115, 0.37335075422604186, 0.37305531388623314], 'original_runing_time': 140.1285092830658, 'euclidean_mimic_result': [3352, 25113, 9248, 12498, 22375, 30399, 19138, 552, 1784, 26222, 11191, 33450, 9264, 11690, 1895, 2531, 26205, 1534, 21267, 7555, 18302, 18435, 31680, 2923, 31601, 13615, 17931, 22174, 3110, 16706, 8671, 25562, 27407, 23712, 25545, 27696, 33468, 4690, 31197, 87, 3987, 4854, 4746, 7792, 28621, 534, 20852, 3256, 3258, 3005], 'euclidean_mimic_score': [3442.6095302816843, 645.2068513331736, 544.1974901388107, 358.09661031291324, 357.3264089518989, 305.25492895336146, 237.0761496911101, 180.980496845144, 175.62239454943273, 148.46279461132957, 131.24503408611326, 124.04672331157506, 108.38968687511661, 100.93046405215499, 99.47320634768892, 90.93598072157525, 86.47764081952134, 80.08044581085555, 74.28622554670994, 73.52281054229661, 69.4014034170889, 69.36700315060641, 66.35405461750162, 63.7515447567333, 63.02115769605342, 62.1367507510716, 59.2158601460549, 58.19574398077866, 56.031586258317844, 53.86781622411728, 53.85838252825938, 50.54559278427927, 47.83373254315723, 47.3452917976601, 46.64849284425917, 45.75161903522025, 42.031121076150036, 41.718355066659306, 41.713644249493875, 40.54405514719955, 40.010772927179005, 39.539176683101246, 35.24727339545526, 34.04319063682366, 33.50366095548613, 33.35654366429834, 32.77217296454692, 32.51983991549848, 31.5196264244602, 30.7262603774722], 'euclidean_reverse_result': [28990, 6474, 10626, 30414, 4865, 26349, 11979, 246, 28706, 7847, 22320, 26713, 14911, 33142, 21254, 18227, 20286, 26403, 24049, 10912, 8805, 25537, 28074, 18089, 10397, 26065, 3770, 26893, 3336, 17954, 22696, 5652, 16044, 24186, 24992, 32932, 15978, 23823, 7533, 29926, 16106, 16667, 7041, 93, 29159, 32564, 22569, 15129, 3384, 30028], 'euclidean_reverse_score': [-22.97627580406525, -23.195796816792463, -24.068164854392776, -24.405068739783793, -24.646369901301647, -24.871245362220154, -25.010857581728747, -25.086822069687315, -26.561389522996954, -26.623843891948066, -26.887014754305618, -26.932149997737906, -30.291826760113572, -30.552623902254155, -31.050469803690792, -32.46528519695183, -32.97369534069934, -34.30489780236081, -34.613709096941506, -34.73575333287013, -39.81999254003172, -41.97914365213964, -42.53703148389172, -47.925361856725374, -49.499683978899654, -57.8782408175273, -59.011631487729744, -60.05057390029238, -62.98461875671101, -65.09640944486864, -72.85652440474978, -75.3802741561702, -79.57767552675782, -81.36547031367222, -81.58163196057801, -82.69222525892623, -108.2219429153361, -108.93365537338167, -125.11227175214397, -135.41616853412765, -137.47123648114422, -156.0709533080156, -187.22001966647144, -188.26460553447808, -200.06682711637544, -216.72737995276657, -270.8943554187116, -492.7274791686317, -860.6234715305114, -1747.6806300182234], 'euclidean_running_time': 39.1226167678833, 'cosine_mimic_result': [3352, 25113, 9248, 12498, 22375, 30399, 19138, 552, 1784, 26222, 11191, 33450, 9264, 11690, 1895, 2531, 26205, 1534, 21267, 7555, 18302, 18435, 31680, 2923, 31601, 13615, 17931, 22174, 3110, 16706, 8671, 25562, 27407, 23712, 25545, 27696, 33468, 4690, 31197, 87, 3987, 4854, 4746, 7792, 28621, 534, 20852, 3256, 3258, 3005], 'cosine_mimic_score': [3442.6095302816843, 645.2068513331736, 544.1974901388107, 358.09661031291324, 357.3264089518989, 305.25492895336146, 237.0761496911101, 180.980496845144, 175.62239454943273, 148.46279461132957, 131.24503408611326, 124.04672331157506, 108.38968687511661, 100.93046405215499, 99.47320634768892, 90.93598072157525, 86.47764081952134, 80.08044581085555, 74.28622554670994, 73.52281054229661, 69.4014034170889, 69.36700315060641, 66.35405461750162, 63.7515447567333, 63.02115769605342, 62.1367507510716, 59.2158601460549, 58.19574398077866, 56.031586258317844, 53.86781622411728, 53.85838252825938, 50.54559278427927, 47.83373254315723, 47.3452917976601, 46.64849284425917, 45.75161903522025, 42.031121076150036, 41.718355066659306, 41.713644249493875, 40.54405514719955, 40.010772927179005, 39.539176683101246, 35.24727339545526, 34.04319063682366, 33.50366095548613, 33.35654366429834, 32.77217296454692, 32.51983991549848, 31.5196264244602, 30.7262603774722], 'cosine_reverse_result': [28990, 6474, 10626, 30414, 4865, 26349, 11979, 246, 28706, 7847, 22320, 26713, 14911, 33142, 21254, 18227, 20286, 26403, 24049, 10912, 8805, 25537, 28074, 18089, 10397, 26065, 3770, 26893, 3336, 17954, 22696, 5652, 16044, 24186, 24992, 32932, 15978, 23823, 7533, 29926, 16106, 16667, 7041, 93, 29159, 32564, 22569, 15129, 3384, 30028], 'cosine_reverse_score': [-22.97627580406525, -23.195796816792463, -24.068164854392776, -24.405068739783793, -24.646369901301647, -24.871245362220154, -25.010857581728747, -25.086822069687315, -26.561389522996954, -26.623843891948066, -26.887014754305618, -26.932149997737906, -30.291826760113572, -30.552623902254155, -31.050469803690792, -32.46528519695183, -32.97369534069934, -34.30489780236081, -34.613709096941506, -34.73575333287013, -39.81999254003172, -41.97914365213964, -42.53703148389172, -47.925361856725374, -49.499683978899654, -57.8782408175273, -59.011631487729744, -60.05057390029238, -62.98461875671101, -65.09640944486864, -72.85652440474978, -75.3802741561702, -79.57767552675782, -81.36547031367222, -81.58163196057801, -82.69222525892623, -108.2219429153361, -108.93365537338167, -125.11227175214397, -135.41616853412765, -137.47123648114422, -156.0709533080156, -187.22001966647144, -188.26460553447808, -200.06682711637544, -216.72737995276657, -270.8943554187116, -492.7274791686317, -860.6234715305114, -1747.6806300182234], 'cosine_running_time': 39.08220076560974}

In [6]:
import json
with open("text.json", 'w') as file:
    json.dump(result, file, indent=4)